In [ ]:
# This document details the circuit design decisions made for the LTC7871 buck converter. The goal here is to provide an easy
# way to evaluate how component values will affect the overall behavior and performance, without having to restrict myself to
# an excel sheet or a SPICE simulation. This is a living document and will be updated as I learn more about the LTC7871 and
# develop the hardware.

: 

In [ ]:
# *****************************************************************************
# System Design Constraints
# *****************************************************************************
Pwr_max = 1000.0 # Maximum power output of the converter in watts.
Eff_min = 0.75 # Estimated minimum efficiency of the converter under typical conditions.
Eff_max = 0.97 # Estimated maximum efficiency of the converter under typical conditions.
Vout_min = 12.0 # Minimum output voltage. Not enough to charge, but could power an inverter?
Vout_max = 56.0 # High enough to charge a 48V battery bank.
Vin_min = 24.0 # Photovoltaic input voltage required to charge a 12v battery bank with some headroom.
Vin_max = 90.0 # Photovoltaic input voltage max of the LTC7871 with some headroom.
Iout_max = Pwr_max / Vout_min # Maximum output current in amps.
Iout_min = Pwr_max / Vout_max # Minimum output current in amps.

: 

In [ ]:
# *****************************************************************************
# Operating Conditions to Evaluate
# *****************************************************************************
# Configurable Fields
VLow = 12.0  # Low side output voltage
VHigh = 56.0  # High side input voltage
dIL = 0.1  # Desired inductor ripple current in Amps
Fosc = 500e3  # Switching frequency in Hz

# Computed Operating Points
op_Iout_max = Pwr_max / VLow  # Maximum output current in Amps

: 

In [ ]:
# *****************************************************************************
# Current Limit Programming
# *****************************************************************************
from enum import Enum

class ILim(Enum):
    """ Enumeration of the current limit programming options for the LTC7871 Table 5 """
    ILIM_0 = 0
    ILIM_1_4 = 1
    ILIM_FLOAT = 2
    ILIM_3_4 = 3
    ILIM_FULL = 4

# Maximum voltage sense values for a DCR type current sense resistor.
# Values are in volts.
VSense_Max_DCR = {
    ILim.ILIM_0: 0.1,
    ILim.ILIM_1_4: 0.2,
    ILim.ILIM_FLOAT: 0.3,
    ILim.ILIM_3_4: 0.4,
    ILim.ILIM_FULL: 0.5
}

# Selection for the current limit programming.
ILim_Select = ILim.ILIM_FULL
VSense_Max = VSense_Max_DCR[ILim_Select]

: 

In [ ]:
# *****************************************************************************
# DCR and Inductor Selection (Figure 3)
# *****************************************************************************
R1 = 10e3
C1 = 0.1e-6
R2 = 49.9e3
C2 = 0.1e-6
L = 10e-6
DCR = 2e-3

# Constraints
act_Vsense = DCR * (op_Iout_max + (dIL / 2.0))
assert act_Vsense < VSense_Max, "The actual voltage sense is greater than the maximum allowed voltage sense."


# Ensure the ripple voltage is less than 2mV
dVsense = (VLow / VHigh) * ((VHigh - VLow) / (R1 * C1 * Fosc))

: 